In [2]:
# Modulos básicos
import numpy as np
import time
#from pylab import imshow
import matplotlib.pyplot as plt
from tqdm import tqdm, tnrange, tqdm_notebook
# Modulo para manejo de fecha
from datetime import datetime, timedelta
# Modulos para astrofisica/solar
import astropy
from sunpy.net import vso
import astropy.units as u
from sunpy.map import Map
from astropy.io import fits # to fix headers
# Custom-made methods and classes for fixing headers
#from lib.CompatMaps import sinehpc_wcs_frame_mapping

# Obtener los datos
Se usa el cliente VSO de SunPy para obtener los datos automáticamente, solo se tienen que cambiar las fechas correspondientes. Fechas de interés para el proyecto son:
* 2011/02/12 a 2011/02/17
* 2012/01/23 a 2012/01/28
* 2013/03/04 a 2013/03/09
* 2014/09/23 a 2013/09/28
* 2015/09/03 a 2015/09/08
* 2016/03/11 a 2016/03/16

In [31]:
# defining datetime range and number of samples 
dates = []  # where the dates pairs are going to be stored
date_start = datetime(2013,3,4,8,0,0)
date_end = datetime(2013,3,9,23,59,59)
date_samples = 200  # Number of samples to take between dates
date_delta = (date_end - date_start)/date_samples  # How frequent to take a sample
date_window = timedelta(minutes=2)
temp_date = date_start
while temp_date < date_end:
    dates.append((str(temp_date),str(temp_date+date_window)))
    temp_date += date_delta 

In [ ]:
# definir instrumento
instrument = 'hmi'
# definir rango de longitud de onda (min,max)
#wavelength = 400*u.nm , 700*u.nm

# Query data - Buscar datos en esas fechas
t = 0
for i in dates:
    tstart, tend = i[0], i[1]
    data_client = vso.VSOClient()
    data_query = data_client.query(vso.attrs.Time(tstart, tend), \
                                   vso.attrs.Instrument(instrument), vso.attrs.Physobs("intensity"))
    print("Found ",len(data_query)," records from ", tstart, " to ", tend)
    print("Time range: ", data_query.time_range())
    print("Size in KB: ", data_query.total_size())
    data_dir = '/home/ivan/projects/Physics/solar/solar-physics-ex/rotation/data/set2/{file}.fits'
    results = data_client.get(data_query, path=data_dir)
    if t%2 == 0:
        time.sleep(30)
    t+=1

Found  3  records from  2013-03-04 08:00:00  to  2013-03-04 08:02:00
Time range:  (datetime.datetime(2013, 3, 4, 8, 0, 28), datetime.datetime(2013, 3, 4, 8, 1, 59))
Size in KB:  0
Found  3  records from  2013-03-04 08:40:47.995000  to  2013-03-04 08:42:47.995000
Time range:  (datetime.datetime(2013, 3, 4, 8, 40, 58), datetime.datetime(2013, 3, 4, 8, 42, 29))
Size in KB:  0
Found  2  records from  2013-03-04 09:21:35.990000  to  2013-03-04 09:23:35.990000
Time range:  (datetime.datetime(2013, 3, 4, 9, 22, 13), datetime.datetime(2013, 3, 4, 9, 22, 59))
Size in KB:  0
